# Installing packages

In [ ]:
!pip install nemoguardrails
!pip install pinecone-client
!pip install pinecone

# RAG


In [58]:
import torch

device = f'cuda:{torch.cuda.current_device()}' if torch.cuda.is_available() else 'cpu'

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
from datasets import load_dataset, Dataset

path = '/content/certibot-data.jsonl'

raw_dataset = load_dataset('json', data_files=path, split='train')
raw_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['topic', 'article'],
    num_rows: 97
})

In [5]:
data = raw_dataset.to_pandas()

data = data.reset_index()
data['context'] = data['topic'] + ': ' + data['article']
data['index'] = data['index'].astype((str))

dataset = Dataset.from_pandas(data)

In [6]:
# checkpoint = "Cohere/Cohere-embed-multilingual-v3.0"
checkpoint = "intfloat/multilingual-e5-large-instruct"
embed_dim = 1024

In [7]:
embedding = HuggingFaceEmbeddings(model_name=checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/128 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/140k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/271 [00:00<?, ?B/s]

In [8]:
import pinecone

api_key = 'a344a187-8b52-422d-97c2-96628ef67ef6'

pc = pinecone.Pinecone(api_key=api_key)

In [9]:
index_name = 'certibot-rag'

In [10]:
import time

# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=embed_dim,
        metric='cosine',
        spec=pinecone.PodSpec('gcp-starter')
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1024,
 'index_fullness': 0.00097,
 'namespaces': {'': {'vector_count': 97}},
 'total_vector_count': 97}

In [11]:
batch_size = 4

In [12]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(0, len(data), batch_size))

for i in range(0, len(data), batch_size):
  end = min(len(data), i + batch_size)
  batch = data[i:end]

  ids = batch['index']
  texts = batch['context']

  embeds = embedding.embed_documents(texts)

  metadata = [{
      'context': x['context'],
      'topic': x['topic']
      } for _, x in batch.iterrows()]

  vectors = list(zip(ids, embeds, metadata))
  index.upsert(vectors=vectors)
  progress_bar.update(1)

  0%|          | 0/25 [00:00<?, ?it/s]

# RAG functions

In [13]:
def retrieve(query:str, top_k:int=5) -> list:
  """
  Embeds a user query, retrieves top_k relevant contexts and returns them for
  use by the LLM.
  """

  query_embed = embedding.embed_query(query)
  res = index.query(vector=query_embed, top_k=top_k, include_metadata=True)
  contexts = [x['metadata']['context'] for x in res['matches']]

  return contexts

In [14]:
res = retrieve("comment souscrire à l'offre free")

In [15]:
res

["Offre Free Mobile: Si vous êtes abonné mobile Forfait Free ou Série Free, vous bénéficiez de tarifs exclusifs sur CertiDeal. Pour profiter de l'offre, il suffit de vous connecter sur votre espace Abonné mobile Free, rubrique Mon compte -> Mon avantage.\nSi vous n'êtes pas encore abonné mobile Free, vous pouvez aussi bénéficier de l'offre exclusive CertiDeal avec Free Pour cela :\nChoisissez votre smartphone incluant les 5 % de remise minimum sur abonnés.certideal.com et valider votre panier sur CertiDeal. Après le paiement, vous serez redirigé automatiquement sur le site mobile Free pour souscrire votre forfait.",
 "Puis-je bénéficier de la remise Free tout au long de l'année?: En souscrivant un forfait mobile Free, vous pourrez bénéficier de la remise de 5% minimum sur l'achat de votre smartphone CertiDeal à tout moment de l'année, dans la limite d'un smartphone par an et par forfait.",
 'Paiement en plusieurs fois: Vous avez la possibilité de payer votre commande en 3 ou 4 fois sur

In [16]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-v0.1"
# model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)


# bnb_config = BitsAndBytesConfig(load_in_4bit=True,
#                                 bnb_4bit_quant_type='nf4',
#                                 bnb_4bit_use_double_quant=True,
#                                 bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_name,
                                            #  quantization_config=bnb_config,
                                             )

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [71]:
def rag(query:str, contexts:list) -> str:
  print('> RAG Called')

  context = ""
  for i, text in enumerate(contexts):
    context += f'*{i}: {text}\n\n'

  prompt = f"""
  Tu es un assistant pour les clients de CertiDeal. CertiDeal vends des smartphones reconditionnés.
  Ci-dessous est une requête d'un utilisateur ainsi que quelques contextes pertinents.
  Réponds à la question en fonction des informations contenues dans ces contexts.
  Les réponses doivent être précises et relativement courtes.
  Si tu ne trouve pas la réponse à la question, dis "Je ne sais pas".

  Contextes:
  {context}

  Question: {query}
  Crée une réponse informative sans recopier la requête. Montre que tu as compris la demande en apportant une réponse précise et pertinente.
  """

  inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
  input_ids = inputs['input_ids'].to(device)
  attention_mask = inputs['attention_mask'].to(device)
  outputs = model.generate(
      input_ids,
      attention_mask=attention_mask,
      temperature=0.5,
      do_sample=True,
      max_new_tokens=256,
      )
  answer = tokenizer.decode(outputs[0].to(device), skip_special_tokens=True)

  return answer

In [72]:
out = rag(query="comment souscrire à l'offre free", contexts=res)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


> RAG Called


In [73]:
out

'<s> \n  Tu es un assistant pour les clients de CertiDeal. CertiDeal vends des smartphones reconditionnés.\n  Tu dois répondre à toutes les questions concernant CertiDeal, ses produits, ses services et ses partenariats.\n\n  Ci-dessous est une requête d\'un utilisateur ainsi que quelques contextes pertinents.\n  Réponds à la question en fonction des informations contenues dans ces contexts.\n  Si tu ne trouve pas la réponse à la question, dis "Je ne sais pas".\n\n  Contextes:\n  Offre Free Mobile: Si vous êtes abonné mobile Forfait Free ou Série Free, vous bénéficiez de tarifs exclusifs sur CertiDeal. Pour profiter de l\'offre, il suffit de vous connecter sur votre espace Abonné mobile Free, rubrique Mon compte -> Mon avantage.\nSi vous n\'êtes pas encore abonné mobile Free, vous pouvez aussi bénéficier de l\'offre exclusive CertiDeal avec Free Pour cela :\nChoisissez votre smartphone incluant les 5 % de remise minimum sur abonnés.certideal.com et valider votre panier sur CertiDeal. Ap

In [65]:
tokenizer.decode(token_ids=out[0].squeeze().to(device))

'<s> \n  Tu es un assistant pour les clients de CertiDeal. CertiDeal vends des smartphones reconditionnés.\n  Tu dois répondre à toutes les questions concernant CertiDeal, ses produits, ses services et ses partenariats.\n\n  Ci-dessous est une requête d\'un utilisateur ainsi que quelques contextes pertinents.\n  Réponds à la question en fonction des informations contenues dans ces contexts.\n  Si tu ne trouve pas la réponse à la question, dis "Je ne sais pas".\n\n  Contextes:\n  Offre Free Mobile: Si vous êtes abonné mobile Forfait Free ou Série Free, vous bénéficiez de tarifs exclusifs sur CertiDeal. Pour profiter de l\'offre, il suffit de vous connecter sur votre espace Abonné mobile Free, rubrique Mon compte -> Mon avantage.\nSi vous n\'êtes pas encore abonné mobile Free, vous pouvez aussi bénéficier de l\'offre exclusive CertiDeal avec Free Pour cela :\nChoisissez votre smartphone incluant les 5 % de remise minimum sur abonnés.certideal.com et valider votre panier sur CertiDeal. Ap